In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('cleand_tweets.csv')
time_stamp = pd.to_datetime(df['time_stamp'])
old_stamp = max(time_stamp)

In [3]:
df = pd.read_csv('tweets_translated.csv')
df['tweet']

0         @F1 No one try to fight vs. Red Bull or Merced...
1         @TheBishF1 @nuerburgring @F1 @virtualstatman A...
2         Boring: Football transfer window  Exciting: F1...
3         @MasonH_07 @F1 Literally when defending two ca...
4         Anyone wanna sponsor a solid 180 euro so i can...
                                ...                        
236228    1968: Born Ricardo Rosset, Brazilian Formula 1...
236229    Shitty no one knows how to race with formula 1...
236230    ((The Most Read)) #Noticias #mundotuerca #form...
236231    💖🐶 'Fabbio' brings all the attitude of formula...
236232    @marca @Carlossainz55 RIDICULOUS .... that's h...
Name: tweet, Length: 236233, dtype: object

In [4]:
df.isnull().sum(axis=0)

id                      0
conversation_id     71877
created_at          71877
date                    2
time                    2
timezone            71877
user_id                 2
username                2
name                    6
place              236190
tweet                   4
language            11378
mentions            11378
urls                11378
photos              11378
replies_count       11378
retweets_count      11378
likes_count         11378
hashtags            11378
cashtags            11378
link                11378
retweet             11378
quote_url          230599
video               11378
thumbnail          196221
near               236233
geo                236233
source             236233
user_rt_id         236233
user_rt            236233
retweet_id         236233
reply_to            11378
retweet_date       236233
translate          236233
trans_src          236233
trans_dest         236233
time_stamp              2
Unnamed: 0.1       101886
Unnamed: 0  

In [5]:
#Droping all coulumns that are empty or with many missing values.
df = df.drop(['thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'retweet_date',
            'translate', 'trans_src', 'trans_dest'], axis=1)
df = df.dropna(subset=['hashtags', 'date', 'tweet'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224853 entries, 0 to 236232
Data columns (total 30 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               224853 non-null  object 
 1   conversation_id  152979 non-null  float64
 2   created_at       152979 non-null  object 
 3   date             224853 non-null  object 
 4   time             224853 non-null  object 
 5   timezone         152979 non-null  float64
 6   user_id          224853 non-null  float64
 7   username         224853 non-null  object 
 8   name             224849 non-null  object 
 9   place            43 non-null      object 
 10  tweet            224853 non-null  object 
 11  language         224853 non-null  object 
 12  mentions         224853 non-null  object 
 13  urls             224853 non-null  object 
 14  photos           224853 non-null  object 
 15  replies_count    224853 non-null  float64
 16  retweets_count   224853 non-null  floa

In [7]:
df = df.drop_duplicates(subset='id')

In [8]:
missing = df[df.tweet.isna()]
if not missing.empty:
    print(missing.index)

In [9]:
df.drop(78396, inplace=True)

In [10]:
df.shape

(224852, 30)

In [11]:
len(df['id'].unique())

224852

In [12]:
df['time_stamp'] = pd.to_datetime(df['time_stamp'])

In [13]:
old_df = df[df['time_stamp'] <= old_stamp]
old_df.shape

(203752, 30)

In [14]:
def replace_formula(tweet):
    t = tweet.replace('formula ', 'f')
    return t

In [15]:
df['tweet'] = df['tweet'].str.lower()

In [16]:
df.tweet

0         @f1 no one try to fight vs. red bull or merced...
1         @thebishf1 @nuerburgring @f1 @virtualstatman a...
2         boring: football transfer window  exciting: f1...
3         @masonh_07 @f1 literally when defending two ca...
4         anyone wanna sponsor a solid 180 euro so i can...
                                ...                        
236228    1968: born ricardo rosset, brazilian formula 1...
236229    shitty no one knows how to race with formula 1...
236230    ((the most read)) #noticias #mundotuerca #form...
236231    💖🐶 'fabbio' brings all the attitude of formula...
236232    @marca @carlossainz55 ridiculous .... that's h...
Name: tweet, Length: 224852, dtype: object

In [17]:
df['tweet'] = df['tweet'].apply(lambda t: replace_formula(str(t)))

## Cleaning Tweets 

In [18]:
def hashtag_removal(r):
    """
    function to remove hashtags from a tweet
    """
    twt=r['tweet'].lower()
    tags=list(r['hashtags'][1:-1].split(','))
    for i in tags:
        i=i.strip()
        i=i[1:-1]
        twt=twt.replace('#'+i,'')
    return twt 
df['clean_tweet']=df.apply(lambda x: hashtag_removal(x),axis=1)

In [19]:
!pip install tweet-preprocessor
import preprocessor as p
#Clening the tweets using tweet-preprocesor
df['clean_tweet']=df['clean_tweet'].apply(lambda x:p.clean(x))

In [20]:
import regex as re
def punctuation_removal(r):
    """
    Removing punctioion from tweet
    """
    ls=re.findall(r'&(\w+);', r)
    for i in ls:
        r=r.replace(f'&{i};','')
    punc = '’’!()-[]{};:’”\,<>./?@#$%^&*_~’’' 
    for ele in r:
         if ele in punc:
            r = r.replace(ele, '')
         return r
df['clean_tweet']=df['clean_tweet'].apply(lambda x:punctuation_removal(x))

In [21]:
#Remove Digits
df['clean_tweet'] = df['clean_tweet'].astype(str).str.replace(' \d+', '')
df['clean_tweet']

0         no one try to fight vs. red bull or mercedes. ...
1         although the season was run under f2 regulatio...
2         boring: football transfer window exciting: f1 ...
3         literally when defending two cars so much bett...
4         anyone wanna sponsor a solid euro so i can liv...
                                ...                        
236228            born ricardo rosset, brazilian f1 driver.
236229    shitty no one knows how to race with f1. they ...
236230    the most read)) bentley celebrates years of co...
236231              'fabbio' brings all the attitude of f1.
236232             ridiculous .... that's how it goes in f1
Name: clean_tweet, Length: 224852, dtype: object

## Tokenization

In [22]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
 return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
df['tokenized_tweet'] = df['clean_tweet'].apply(lambda x:lemmatize_text(x))
df['tokenized_tweet']

[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /home/repl/nltk_data...


0         [no, one, try, to, fight, v, ., red, bull, or,...
1         [although, the, season, wa, run, under, f2, re...
2         [boring, :, football, transfer, window, exciti...
3         [literally, when, defending, two, car, so, muc...
4         [anyone, wanna, sponsor, a, solid, euro, so, i...
                                ...                        
236228    [born, ricardo, rosset, ,, brazilian, f1, driv...
236229    [shitty, no, one, know, how, to, race, with, f...
236230    [the, most, read, ), ), bentley, celebrates, y...
236231    [', fabbio, ', brings, all, the, attitude, of,...
236232       [ridiculous, ..., that's, how, it, go, in, f1]
Name: tokenized_tweet, Length: 224852, dtype: object

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

new_index = [i for i in range(len(df.index))]
tweets = df['clean_tweet'].values[new_index]
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(tweets)

sequences = tokenizer.texts_to_sequences(tweets)
padded = pad_sequences(sequences)


2022-08-03 16:56:04.234063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-03 16:56:04.234092: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [24]:
df_labeling = df[:100000]
df_val = df[100000:105000]

In [25]:
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.2/768.2 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 21.9 MB/s eta 0:00:00


In [26]:
from textblob import TextBlob
testimonial = TextBlob("Max drove great")
testimonial.sentiment
testimonial.sentiment.polarity

0.8

In [27]:
df_labeling.reset_index(inplace=True)

In [28]:
df_labeling.drop('index', axis=1, inplace=True)

In [29]:
df_labeling['label'] = ''

In [30]:
df_part = df_labeling[126:151]

In [31]:
df_labeling.drop_duplicates(subset='tweet', inplace=True)

In [32]:
for index, tweet in df_part.tweet.iteritems():
    print(tweet)
    lab = input('Positive or negative')
    df_labeling['label'][index] = lab

schumacher: 'battle with hamilton shouldn't have happened' #grandprix #gptoday #formula1 #f1 #f1news #motorsport  https://t.co/ntkx2gupzn


KeyboardInterrupt: Interrupted by user

In [ ]:
df_labeling['label'][101:126]

In [ ]:
df_labeling['label'].value_counts()

In [124]:
df_labeling.to_csv('labeled_tweets.csv')

In [ ]:
df_labeling ['label'] = ''
for i,x in df['clean_tweet'].iteritems():
    label = TextBlob(x)
    df_labeling ['label'][i] = label.sentiment.polarity
    print("Index: ", i ,  "label" , label.sentiment.polarity)

**Stop Words removal**

In [33]:
stop_words = set(stopwords.words('english'))
df['tokenized_tweet'] = df['tokenized_tweet'].apply(lambda x: [item for item in x if item not in stop_words])

In [34]:
df = df.sort_values('date')
df.shape
df['tweet'][0]

'@f1 no one try to fight vs. red bull or mercedes. everyone wake up only vs. ferrari. we are most hated team of the grid for no reason.'

In [35]:
df.to_csv('cleand_tweets.csv', index=False)

In [55]:
date_range = list(df.date.unique())

In [95]:
token_dict = {}
date_dict = {}
token_list = []
totals_list = []
for date in date_range:
    df_range = df[df.date == date]
    for index, tokens in df_range['tokenized_tweet'].items():
        if tokens:
            for token in tokens:

                if str(token) in token_dict:
                    token_dict[str(token)] += 1
                else:
                    token_list.append(token)
                    token_dict[str(token)] = 1
    
    date_dict[date] = token_dict.values()
    token_dict = dict.fromkeys(token_dict, 0)
# token_series = pd.Series(token_dict)
date_dict['words'] = token_dict.keys()
date_dict.keys()

dict_keys(['2022-07-14', '2022-07-15', '2022-07-16', '2022-07-17', '2022-07-18', '2022-07-19', '2022-07-20', '2022-07-23', '2022-07-24', '2022-07-25', '2022-07-26', '2022-07-27', '2022-07-28', '2022-07-29', '2022-07-30', '2022-07-31', '2022-08-01', '2022-08-02', '2022-08-03', 'words'])

In [97]:
list_len = []
for l in date_dict.keys():
    list_len.append(len(date_dict[l]))
list_len

[2752,
 9168,
 12024,
 14429,
 16914,
 19106,
 20414,
 21596,
 25581,
 28578,
 30518,
 32194,
 35595,
 38606,
 43165,
 49588,
 52333,
 54902,
 55454,
 55454]

In [108]:
max_len = max(list_len)
index = 0
for totals in date_dict.keys():
    zeros_to_add = max_len - list_len[index]
    zeros_list = [0] * zeros_to_add
    date_dict[totals] = list(date_dict[totals]) + zeros_list 
    index += 1

In [109]:
list_len = []
for l in date_dict.keys():
    list_len.append(len(date_dict[l]))
list_len

[55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454,
 55454]

In [112]:
date_totals = pd.DataFrame.from_dict(date_dict)

In [113]:
date_totals.to_csv('totals_date.csv')

In [37]:
token_series.sort_values(ascending=False)
token_series.drop(['.', ',', '!', ':', '?', '-', '"', '...', "'", 'wa'], inplace=True)
token_series.sort_values(ascending=False)[:20]

f1           70137
live         20801
race         16493
None         15129
grand        14001
prix         13864
driver       13739
like         11746
car          11322
ferrari      10407
ha           10185
max           9795
team          9651
stream        9407
hungarian     9238
one           8474
watch         8422
1             8380
year          8266
time          8017
dtype: int64

In [38]:
token_series.to_csv('token_tweets.csv', index=False)